In [3]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
loader = PyPDFLoader('files/solar.pdf')

docs = loader.load()

In [6]:
recur_split = RecursiveCharacterTextSplitter(
	chunk_size=500,
	chunk_overlap=50,
	separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(docs)

In [8]:
from langchain_openai import  OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()

In [9]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(
	documents=documents,
	embedding=embeddings_model
)

retriever = vectorstore.as_retriever(search_type="mmr")

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser 

model = ChatOpenAI()

template_str = """Responda as seguintes perguntas do usuário utilizando apenas o contexto fornecido.
Contexto: {contexto}
Pergunta: {pergunta}
"""

template = ChatPromptTemplate.from_template(template_str)

output_parser = StrOutputParser()

In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough 

setup_and_retrievel = RunnableParallel(
	{
		"pergunta": RunnablePassthrough(),
		"contexto": retriever
	}
)

chain = setup_and_retrievel | template | model | output_parser

In [14]:
chain.invoke("Qual o contado do help desk?")

'O contato do HelpDesk é (85) 3101-3430 ou helpdesk@defensoria.ce.def.br.'

In [15]:
from langchain_openai import OpenAI 
from langchain.prompts import PromptTemplate 

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template("Resuma o seguinte texto: {texto}")

chain_pequena = prompt | llm 
chain_pequena.invoke({"texto": "Oi, eu sou o lucas"})

'\n\nEu sou o Lucas.'

In [16]:
chain_pequena.invoke({"texto": "um texto grande" * 1000})

BadRequestError: Error code: 400 - {'error': {'message': "Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", 'type': 'invalid_request_error', 'param': 'prompt', 'code': 'invalid_prompt'}}

In [18]:
from langchain_openai import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser 

model = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("Resuma o seguinte texto: {texto}")
chain_grande = prompt | model | StrOutputParser 
chain_grande.invoke({"texto": "Teste prompt" * 1000})

BadRequestError: Error code: 400 - {'error': {'message': "Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", 'type': 'invalid_request_error', 'param': 'prompt', 'code': 'invalid_prompt'}}